<a href="https://colab.research.google.com/github/RyanSolving/PainPoints_Extracter/blob/main/WWReviews_DataCrawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# `Reviews database Project`

In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, parse_qs, urlencode, unquote, quote
import json
import time
import random
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
import sqlite3
import uuid
from datetime import datetime

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Connect database google SQL

### Create table stores_list

In [ ]:
conn = sqlite3.connect('/content/drive/MyDrive/Projects/Woolies Reviews/wwreviews.db')
cursor = conn.cursor()

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Projects/Woolies Reviews/store_list.csv')

In [ ]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS stores_list (
    Division TEXT NOT NULL,
    StoreNo INTEGER NOT NULL,
    Name TEXT,
    AddressLine1 TEXT,
    AddressLine2 TEXT,
    Suburb TEXT,
    State TEXT,
    Postcode TEXT,
    Latitude REAL,
    Longitude REAL,
    GeoLevel TEXT,
    Distance REAL,
    Phone TEXT,
    GMTZone TEXT,
    IsOpen BOOL,
    PartnerUrl REAL,
    TradingHours TEXT,
    Facilities TEXT,
    CategorisedFacilities TEXT,
    NearbyPartners REAL,
    Timestamp DATETIME,
    review_initial_url TEXT NOT NULL,
    review_nextpage_url TEXT NOT NULL,
    PRIMARY KEY (StoreNo)
)
''')

In [ ]:
df.to_sql('stores_list',
          conn,
          if_exists='replace',
          index=False)

1393

### Create reviews_list

In [ ]:
conn.execute('DROP TABLE IF EXISTS store_reviews')

conn.execute('''
CREATE TABLE store_reviews (
    ReviewID TEXT NOT NULL,
    StoreNo INTEGER NOT NULL,
    StoreName TEXT,
    UserUrl TEXT,
    UserName TEXT,
    LocalGuideBadges TEXT,
    ReviewTextSnippet TEXT,
    ReviewTextFull TEXT,  -- Add missing comma here
    Rating REAL,          -- Fixed the missing comma issue
    TimeReview TEXT,
    Timestamp DATETIME,
    PRIMARY KEY (ReviewID),
    FOREIGN KEY (StoreNo) REFERENCES stores_list(StoreNo)
)
''')
conn.commit()

## Data Crawling

In [ ]:
def extract_reviews_from_html(html_content):
    """Extract reviews from initial HTML content using BeautifulSoup."""
    soup = BeautifulSoup(html_content, 'html.parser')

    review_containers = soup.find_all('div', class_='gws-localreviews__google-review')
    all_reviews = []  # Initialize an empty list to store all reviews

    for review_container in review_containers:
        review_data = {}  # Initialize an empty dictionary for each review

        user_element = review_container.find('a', style='cursor:pointer')
        if user_element:
            review_data["UserUrl"] = user_element.get('href')
            img_element = user_element.find('img')
            review_data["UserName"] = img_element.get('alt') if img_element else ""
            review_data["LocalGuideBadges"] = ','.join([a.text for a in review_container.find_all('span', class_="QV3IV")])

        text_container = review_container.find('div', class_='Jtu6Td')
        if text_container:
            snippet_element = text_container.find('span', class_='review-snippet')
            full_text_element = text_container.find('span', class_='review-full-text')
            review_data["ReviewTextSnippet"] = snippet_element.text if snippet_element else ""

            if full_text_element:
              review_data["ReviewTextFull"] = full_text_element.text
            else:
              review_data["ReviewTextFull"] = text_container.text
        ratings = review_container.find('span',class_="z3HNkc")
        if ratings:
            review_data["Rating"] = str(ratings.get('aria-label')).replace("Rated ", "").replace(" out of 5,", "")

        time = review_container.find('span', class_="dehysf")
        if time:
          review_data["TimeReview"] = time.text

        all_reviews.append(review_data) # Append the dictionary to the list

    return pd.DataFrame(all_reviews) # Return the final dataframe made from all reviews

In [ ]:
def get_next_page_token(html_content):
  """Get the next page token from the HTML content"""
  soup = BeautifulSoup(html_content, 'html.parser')
  reviews_block = soup.find('div',class_='gws-localreviews__general-reviews-block')
  if reviews_block:
    return reviews_block.get('data-next-page-token')
  return None

In [ ]:
def construct_async_url_orginal(base_url, initial_url, next_page_token):
     """Construct an async URL using parameters from original url.

      Args:
        base_url: The base url from google api
        initial_url: Initial url that contained the review data.
        next_page_token: The next page token for the next page

      Returns:
         url to load more reviews.
     """
     parsed_url = urlparse(initial_url)
     query_params = parse_qs(parsed_url.query)

     async_param_string = query_params.get('async', [''])[0]
     if not async_param_string:
        return None

    # Split at the first colon
     async_params = dict(item.split(':', 1) for item in async_param_string.split(','))

     if next_page_token:
        async_params['next_page_token'] = next_page_token

     full_params = urlencode(async_params) # Remove quote_via=urlencode
     full_params = unquote(full_params)

     return f"{base_url}?{full_params}"

In [ ]:
def construct_async_url(next_page_url, next_page_token):
  """Construct url from example of next_page_url and next_page_token
  Args:
    next_page_url: string, next page url from google
    next_page_token: string, next page token from google

  Returns:
    url to load more reviews.
  """
  #Split into based_url and update_url
  split_url = next_page_url.split(',')
  based_url = split_url[0]
  update_token_url = split_url[1:]
  dict_params = dict(item.split(':') for item in update_token_url)
  dict_params['next_page_token'] = quote(next_page_token)
  dict_params = [f'{key}:{value}' for key,value in dict_params.items()]
  dict_params = ','.join(dict_params)
  return f"{based_url},{dict_params}"


In [ ]:
def load_more_reviews(async_url):
  """Load more reviews from the given async URL.
  Args:
    async_url: string, async url to load more reviews.

  Returns:
    HTML content
  """
  headers = {"User-Agent": "Mozilla/5.0"}
  response = requests.get(async_url, headers=headers)

  if response.status_code == 200:
    return response.text
  else:
    print(f"Failed to load more reviews. Status code: {response.status_code}")
    return None

In [ ]:
# extract_reviews_from_html, get_next_page_token, construct_async_url, load_more_reviews

def crawl_reviews_async(html_content, next_page_url):
    current_html = html_content
    total_reviews = pd.DataFrame(columns=['StoreName','UserUrl', 'UserName', 'LocalGuideBadges', 'ReviewTextSnippet', 'ReviewTextFull', 'Rating', 'TimeReview'])

    while True:  # This is a loop that keeps going until there's no next page
        reviews = extract_reviews_from_html(current_html)

        # Append reviews to the DataFrame
        total_reviews = pd.concat([total_reviews, reviews], ignore_index=True)

        # Get next page token
        next_page_token = get_next_page_token(current_html)

        # If there's no next page token, exit the loop
        if not next_page_token:
            break

        # Construct async URL for the next page
        async_url = construct_async_url(next_page_url, next_page_token)

        # Load the next page content
        if async_url:
            new_html = load_more_reviews(async_url)
            if new_html:
                current_html = new_html
            else:
                break
        else:
            break

        # Sleep between requests to avoid overwhelming the server
        time.sleep(random.uniform(4, 6))

    return total_reviews

## Prepare for crawling

In [ ]:
query_2 = """
SELECT COUNT(DISTINCT StoreNo)
FROM stores_list
WHERE State = 'VIC' AND Division = 'SUPERMARKETS'
"""
cursor.execute(query_2).fetchall()

[(272,)]

In [ ]:
query = 'SELECT StoreNo, review_initial_url, review_nextpage_url FROM stores_list WHERE State="VIC" and Division="SUPERMARKETS"'
cursor = conn.execute(query)
review_links = cursor.fetchall()

In [ ]:
review_links[0]

(3304,
 'https://www.google.com/async/reviewDialog?vet=10ahUKEwj37eCRmfmKAxUrSWwGHRomMCsQlIwOCKcG..i&ei=KW-IZ7e1M6uSseMPmszA2QI&opi=89978449&rlz=1C1FKPE_en-GBAU1093AU1093&yv=3&cs=0&async=feature_id:0x6ad642ca44bc9591%3A0x18895914c0fe118b,review_source:All%20reviews,sort_by:qualityScore,is_owner:false,filter_text:,associated_topic:,next_page_token:,async_id_prefix:,_pms:s,_fmt:pc,_basejs:%2Fxjs%2F_%2Fjs%2Fk%3Dxjs.s.en_GB.imokO5pDFSw.2018.O%2Fam%3DAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAgAAAAAAAAAAACAABQEIAAAAAABQAAgAAAAAAAAAABIAAAAAAEACASBASAAAgAAAAAAAsAAAAAgsAIEBAAAAAAAAAAIAAACAQARgv_84AAAAAAAAAAAAIABEAAAAAADgAgAgCGo3QAAAAACAAAAAAAgAAAAEAAQAAAAUAAAAQAAAAAAIAAAAAAAAAAgAAAAAAQD6QwAAAAAAAAAAAAAAAgAAAAAAaIACAAL4AQAAAAAAgAMAAACAAAEAAMfAAAQAAAAAAADgHgAeDwiHFBYAAAAAAAAAAAAAAAAQgATBHEh_QQACAAAAAAAAAAAAAAAAAABIETRxuQFAAQ%2Fdg%3D0%2Fbr%3D1%2Frs%3DACT90oEmcrTwDGRBHFigA4Hte2jSNiyLXQ,_basecss:%2Fxjs%2F_%2Fss%2Fk%3Dxjs.s.7eaQE2ygUs0.L.B1.O%2Fam%3DAJA6BAgBAAAgAACAEAAqAAgAAAAAAAAAAAAA

In [ ]:
#Extract list of successful added reviews stores
distinct_storeno = conn.execute('''
select distinct StoreNo
from reviews_list
''').fetchall()
distinct_storeno = [item[0] for item in distinct_storeno]

In [ ]:
conn.execute('''SELECT
	count(distinct StoreNo)
FROM reviews_list ''').fetchall()

[(272,)]

In [ ]:
distinct_storeno

[3304,
 3450,
 3438,
 3028,
 3452,
 3711,
 8281,
 3089,
 3163,
 3813,
 3762,
 3453,
 8285,
 3332,
 3306,
 3311,
 3779,
 3398,
 8382,
 3968,
 3195,
 3427,
 3579,
 3607,
 3389,
 3362,
 3325,
 3169,
 3188,
 8827,
 3178,
 3094,
 3403,
 3999,
 3139,
 3411,
 3576,
 3135,
 3307,
 3431,
 3138,
 3115,
 3855,
 3267,
 3391,
 3384,
 3079,
 3964,
 3162,
 3197,
 3221,
 8824,
 3121,
 3096,
 3292,
 8972,
 8823,
 3145,
 3127,
 3353,
 3153,
 3672,
 3117,
 3298,
 8785,
 3157,
 3136,
 3186,
 3291,
 3205,
 3118,
 3155,
 3202,
 3110,
 8826,
 3317,
 3368,
 3340,
 3174,
 3060,
 3209,
 3140,
 3329,
 3122,
 3295,
 3327,
 3235,
 3181,
 3179,
 3128,
 3150,
 3454,
 3147,
 3171,
 3336,
 3763,
 3301,
 3283,
 3802,
 3969,
 3173,
 3289,
 3806,
 3227,
 3055,
 3062,
 3182,
 3187,
 3183,
 3129,
 3392,
 3355,
 3850,
 3399,
 3903,
 3809,
 3146,
 3192,
 3960,
 3149,
 3159,
 3130,
 3109,
 3204,
 3285,
 3063,
 3670,
 3189,
 3282,
 3061,
 3343,
 3223,
 3141,
 3091,
 3354,
 3151,
 3371,
 3216,
 3805,
 3281,
 3382,
 3108,
 3337,

In [ ]:
logs = pd.read_csv('/content/drive/MyDrive/Projects/Woolies Reviews/crawling_logs.csv')

In [ ]:
for store_id, initial_url, next_page_url in review_links:
  if str(store_id) in [str(x) for x in distinct_storeno]: #check if store already exist, convert store_id to string to ensure proper type comparison
    print(f'Skipping {store_id} since it has been processed previously')
    logs = pd.concat([logs, pd.DataFrame([{'store': store_id, 'status': 'skipped'}])], ignore_index=True)
    continue
  try:
    headers = {"User-Agent": "Mozilla/5.0"}
    params = {
    "gl": "US",  # Geolocation: United States
    "hl": "en"   # Language: English
    }
    respond = requests.get(initial_url, headers=headers, params = params)
    respond.raise_for_status() # raise an exception for 4xx and 5xx errors
    initial_html = respond.content
    output = crawl_reviews_async(initial_html, next_page_url)

    if output.empty:
      logs = logs.append({'store': store_id, 'status': 'no_reviews'}, ignore_index=True)
      continue

    output['StoreNo'] = store_id
    output['StoreName'] = BeautifulSoup(initial_html, 'html.parser').find('div',class_='Lhccdd').find('div').text
    output['ReviewID'] = [str(uuid.uuid4()) for _ in range(len(output))]
    output['Timestamp'] = pd.to_datetime(datetime.now())

    # Convert Rating into integer
    output['Rating'] = output['Rating'].str[0].astype(int)

    #Reoder columns
    new_order = ['ReviewID','StoreNo','StoreName','UserUrl','UserName','LocalGuideBadges','ReviewTextSnippet','ReviewTextFull','Rating','TimeReview','Timestamp']
    output = output[new_order]

    #inserting the df to SQL
    output.to_sql('reviews_list',
                conn,
                if_exists='append',
                index=False)
    conn.commit()

    time.sleep(random.uniform(2, 3))
    # Correct way to append a new row to logs
    logs = pd.concat([logs, pd.DataFrame([{'store': store_id, 'status': 'success'}])], ignore_index=True)
    print(f'Success added reviews of store_id {store_id} at {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}')

  except Exception as e:
    #append failure to logs
    logs = pd.concat([logs, pd.DataFrame([{'store': store_id, 'status': 'fail'}])], ignore_index=True)
    print(f"Error processing store {store_id}: {e}")

conn.commit() # Save the changes
conn.close()
logs.to_csv('/content/drive/MyDrive/Projects/Woolies Reviews/crawling_logs.csv', index=False)

print(logs)
print("Reviews processed and added to the database")

Skipping 3304 since it has been processed previously
Skipping 3450 since it has been processed previously
Skipping 3438 since it has been processed previously
Skipping 3028 since it has been processed previously
Skipping 3452 since it has been processed previously
Skipping 3711 since it has been processed previously
Skipping 8281 since it has been processed previously
Skipping 3089 since it has been processed previously
Skipping 3163 since it has been processed previously
Skipping 3813 since it has been processed previously
Skipping 3762 since it has been processed previously
Skipping 3453 since it has been processed previously
Skipping 8285 since it has been processed previously
Skipping 3332 since it has been processed previously
Skipping 3306 since it has been processed previously
Skipping 3311 since it has been processed previously
Skipping 3779 since it has been processed previously
Skipping 3398 since it has been processed previously
Skipping 8382 since it has been processed prev

In [ ]:
conn.commit() # Save the changes
conn.close()
logs.to_csv('/content/drive/MyDrive/Projects/Woolies Reviews/crawling_logs.csv', index=False)


In [ ]:
(initial_url, next_page_url)=conn.execute('''
Select review_initial_url, review_nextpage_url
From stores_list
Where StoreNo = 3855
''').fetchall()[0]

In [ ]:
df.query('StoreNo == c and Division=="SUPERMARKETS"')[['review_initial_url','review_nextpage_url']].values

array([['https://www.google.com/async/reviewDialog?vet=10ahUKEwjYnOvvxvmKAxUhd2wGHTDWBFYQlIwOCPgF..i&ei=Hp-IZ9iVLaHuseMPsKyTsAU&opi=89978449&rlz=1C1FKPE_en-GBAU1093AU1093&sca_esv=bcbd4b002db38cd9&yv=3&cs=0&async=feature_id:0x6ad438e8d754b489%3A0xf4867df92ae95f43,review_source:All%20reviews,sort_by:qualityScore,is_owner:false,filter_text:,associated_topic:,next_page_token:,async_id_prefix:,_pms:s,_fmt:pc,_basejs:%2Fxjs%2F_%2Fjs%2Fk%3Dxjs.s.en_GB.ZIq5YyAIbUU.2018.O%2Fam%3DAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAgAAAAAAAAAAACAABQEIAAAAAABQAAgAAAAAAAAAABIAAAAAAEACASBASAAAgAAAAAAAsAAAAAgsAIEBAAAAAAAAAAIAAACAQARgv_84AAAAAAAAAAAAIABEAAAAAADgAgAgCGo3QAAAAACAAAAAAAgAAAAEAAQAAAAUAAAAQAAAAAAIAAAAAAAAAAgAAAAAAQD6QwAAAAAAAAAAAAAAAgAAAAAAaIACAAL4AQAAAAAAgAMAAACAAAEAAMfAAAQAAAAAAADgHgAeDwiHFBYAAAAAAAAAAAAAAAAQgATBHEh_QQACAAAAAAAAAAAAAAAAAABIETRxuQFAAQ%2Fdg%3D0%2Fbr%3D1%2Frs%3DACT90oGlBOIjHcpfgjtPy3FmV8jc-lwiMw,_basecss:%2Fxjs%2F_%2Fss%2Fk%3Dxjs.s.9Bth82x0PFM.L.B1.O%2Fam%3DAJA6BAgBAAA

In [ ]:
initial_url, next_page_url

('https://www.google.com/async/reviewDialog?vet=10ahUKEwjp-ue5vfmKAxWQSGcHHf1KA3cQlIwOCMUF..i&ei=PZWIZ6noH5CRnesP_ZWNuAc&opi=89978449&rlz=1C1FKPE_en-GBAU1093AU1093&sca_esv=bcbd4b002db38cd9&yv=3&cs=0&async=feature_id:0x6ad65c0c80ecdafd%3A0x4a9bb89e7998dc72,review_source:All%20reviews,sort_by:qualityScore,is_owner:false,filter_text:,associated_topic:,next_page_token:,async_id_prefix:,_pms:s,_fmt:pc,_basejs:%2Fxjs%2F_%2Fjs%2Fk%3Dxjs.s.en_GB.imokO5pDFSw.2018.O%2Fam%3DAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAgAAAAAAAAAAACAABQEIAAAAAABQAAgAAAAAAAAAABIAAAAAAEACASBASAAAgAAAAAAAsAAAAAgsAIEBAAAAAAAAAAIAAACAQARgv_84AAAAAAAAAAAAIABEAAAAAADgAgAgCGo3QAAAAACAAAAAAAgAAAAEAAQAAAAUAAAAQAAAAAAIAAAAAAAAAAgAAAAAAQD6QwAAAAAAAAAAAAAAAgAAAAAAaIACAAL4AQAAAAAAgAMAAACAAAEAAMfAAAQAAAAAAADgHgAeDwiHFBYAAAAAAAAAAAAAAAAQgATBHEh_QQACAAAAAAAAAAAAAAAAAABIETRxuQFAAQ%2Fdg%3D0%2Fbr%3D1%2Frs%3DACT90oEmcrTwDGRBHFigA4Hte2jSNiyLXQ,_basecss:%2Fxjs%2F_%2Fss%2Fk%3Dxjs.s.7eaQE2ygUs0.L.B1.O%2Fam%3DAJA6BAgBAAAgAACAEA

In [ ]:
headers = {"User-Agent": "Mozilla/5.0",
           "Accept-Language": "en-US,en;q=0.9",  # Explicitly request English
           "Content-Language": "en-US"}

params = {
    "gl": "US",  # Geolocation: United States
    "hl": "en"   # Language: English
}
respond = requests.get(initial_url, headers=headers, params = params)
respond.raise_for_status() # raise an exception for 4xx and 5xx errors
initial_html = respond.content
output = crawl_reviews_async(initial_html, next_page_url)

output['StoreNo'] = 3399
output['ReviewID'] = [str(uuid.uuid4()) for _ in range(len(output))]
output['Timestamp'] = pd.to_datetime(datetime.now())
new_order = ['ReviewID','StoreNo','StoreName','UserUrl','UserName','LocalGuideBadges','ReviewTextSnippet','ReviewTextFull','Rating','TimeReview','Timestamp']
output = output[new_order]
output

,ReviewID,StoreNo,StoreName,UserUrl,UserName,LocalGuideBadges,ReviewTextSnippet,ReviewTextFull,Rating,TimeReview,Timestamp
0,9f1ac61b-0399-4da3-8454-a616927be66e,3399,Woolworths Niddrie,/url?q=https://www.google.com/maps/contrib/115...,Julie,,"Woolworths Niddrie has gone downhill big time,...","Woolworths Niddrie has gone downhill big time,...",1.0,a month ago,2025-01-20 23:36:03.589461
1,4ec9f2fe-50bd-460a-8f1a-b6caa76c461a,3399,Woolworths Niddrie,/url?q=https://www.google.com/maps/contrib/106...,Mark B,,I am thrilled to share my experience with Char...,I am thrilled to share my experience with Char...,5.0,2 months ago,2025-01-20 23:36:03.589461
2,a1268d88-c694-4174-8de0-c617bf9c0c31,3399,Woolworths Niddrie,/url?q=https://www.google.com/maps/contrib/108...,Mc Melon,,,I’m writing this to commend Jacob the Nightfil...,5.0,5 months ago,2025-01-20 23:36:03.589461
3,e22c8641-44a2-44bf-90e8-d23fb67f8d5b,3399,Woolworths Niddrie,/url?q=https://www.google.com/maps/contrib/100...,Jordan James,,,ran into a strapping man named ethan the other...,5.0,a month ago,2025-01-20 23:36:03.589461
4,dee5ddc1-47e7-4f12-b9c7-fd33d6c0b2ac,3399,Woolworths Niddrie,/url?q=https://www.google.com/maps/contrib/105...,Kerry Jordan,Local Guide,Drove especially to Niddrie because online sta...,Drove especially to Niddrie because online sta...,1.0,3 months ago,2025-01-20 23:36:03.589461
...,...,...,...,...,...,...,...,...,...,...,...
355,1a022387-5c00-4695-9a11-0698dc63155d,3399,NaN,/url?q=https://www.google.com/maps/contrib/117...,Ruchika Bhandari,Local Guide,,,5.0,6 years ago,2025-01-20 23:36:03.589461
356,cf476b34-a0de-4df0-ba2a-e03364b2c760,3399,NaN,/url?q=https://www.google.com/maps/contrib/117...,Jack Showler,,,,5.0,6 years ago,2025-01-20 23:36:03.589461
357,20b8005a-7011-44cf-ba8d-d5a624fbbb06,3399,NaN,/url?q=https://www.google.com/maps/contrib/103...,Tony Siu,,,,3.0,6 years ago,2025-01-20 23:36:03.589461
358,9e0f3d4e-64f1-4e14-8624-f2d75038abce,3399,NaN,/url?q=https://www.google.com/maps/contrib/117...,Lidija Krčmar,Local Guide,,,3.0,7 years ago,2025-01-20 23:36:03.589461


In [ ]:
#save_output
with open('output.json', 'w') as f:
  json.dump(output, f, indent=4)